In [1]:
import torch
from epsilon_transformers.training.configs.model_configs import RawModelConfig
from epsilon_transformers.training.configs.training_configs import ProcessDatasetConfig
from pathlib import Path
import json

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
checkpoint_path = Path("/workspaces/cure/compmech-models/models/54qc0vyb_mess3-0.05-0.85/")
weights = torch.load(checkpoint_path / "6406400.pt")

In [5]:
with open(checkpoint_path / "train_config.json", "r") as f:
    required_fields = ['d_vocab', 'd_model', 'n_ctx', 'd_head', 'n_heads', 'n_layers']
    cfg_dict = {k: v for k, v in json.load(f).items() if k in required_fields}
    cfg_dict['n_head'] = cfg_dict.pop('n_heads')
    cfg_dict['d_mlp'] = 4 * cfg_dict['d_model']
    train_config = RawModelConfig(**cfg_dict)

model = train_config.to_hooked_transformer(device=torch.device("cuda"))
model.load_state_dict(weights)

<All keys matched successfully>

In [11]:
dataset_config = ProcessDatasetConfig(
    process='Mess3',
    process_params={'x': 0.5, 'a': 0.85},
    batch_size=64,
    num_tokens=50000000,
    test_split=0.00000000015
)

loader = dataset_config.to_dataloader(sequence_length=cfg_dict["n_ctx"], train=True)

In [12]:
for inputs, labels in loader:
    a = model.run_with_cache(inputs)
    break

In [8]:
a[0]

tensor([[[-0.5166,  0.8183, -0.4697],
         [-0.4327,  0.3262,  0.8449],
         [-0.6491,  0.9016, -0.1664],
         ...,
         [-0.3388, -0.8634,  0.5081],
         [-0.4820,  0.3358, -0.0412],
         [-0.6331,  1.0292, -0.3222]],

        [[-0.7741, -0.8289,  0.6545],
         [ 0.2944, -1.1725, -0.1725],
         [ 0.4211, -1.4822, -1.1416],
         ...,
         [ 0.1895, -0.3285, -1.2546],
         [-0.1007, -0.4194,  0.3348],
         [ 0.2354, -1.4510, -0.9999]],

        [[-0.7741, -0.8289,  0.6545],
         [ 0.2944, -1.1725, -0.1725],
         [-0.1597,  0.0259, -0.5297],
         ...,
         [-0.4330, -0.2069,  0.5364],
         [-0.6322,  0.7504, -0.1035],
         [ 0.0478, -0.5892, -0.9713]],

        ...,

        [[-0.7741, -0.8289,  0.6545],
         [ 0.2944, -1.1725, -0.1725],
         [-0.4036, -0.9245,  0.8091],
         ...,
         [-0.4995,  0.2482, -0.0373],
         [-0.7463, -0.4040,  0.9230],
         [-0.7491,  0.7890,  0.0568]],

        [[